In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%config InlineBackend.figure_format = 'retina'


from fastai.vision import *
from fastai.vision.data import ImageDataLoaders
from fastai.metrics import error_rate
from pathlib import Path
from glob2 import glob
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import os
import zipfile as zf
import shutil
import re
import seaborn as sns
import random

files = zf.ZipFile("atik-veriler.zip",'r')
files.extractall()
files.close()

os.listdir(os.path.join(os.getcwd(),"atik-veriler"))



def split_indices(folder,seed1,seed2):    
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    random.seed(seed1)
    train = random.sample(list(range(1,n+1)),int(.5*n))

    remain = list(set(full_set)-set(train))

    random.seed(seed2)
    valid = random.sample(remain,int(.5*len(remain)))
    test = list(set(remain)-set(valid))
    
    return(train,valid,test)


def get_names(waste_type,indices):
    file_names = [waste_type+str(i)+".jpg" for i in indices]
    return(file_names)    

def move_files(source_files,destination_folder):
    for file in source_files:
        shutil.move(file,destination_folder)


## yollar train valid vb olacak
subsets = ['train','valid']
waste_types = ['cardboard','glass','metal','paper','plastic','trash']

## train,valid icin atik turu dosyalarini gonderir 
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data',subset,waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)
            
if not os.path.exists(os.path.join('data','test')):
    os.makedirs(os.path.join('data','test'))
            
## zipdeki verileri hedef folder`lara yonlendirme
for waste_type in waste_types:
    source_folder = os.path.join('atik-veriler',waste_type)
    train_ind, valid_ind, test_ind = split_indices(source_folder,1,1)
    
    ## verileri train`e gonder
    train_names = get_names(waste_type,train_ind)
    train_source_files = [os.path.join(source_folder,name) for name in train_names]
    train_dest = "data/train/"+waste_type
    move_files(train_source_files,train_dest)
    
    ## verileri valid`e gonder
    valid_names = get_names(waste_type,valid_ind)
    valid_source_files = [os.path.join(source_folder,name) for name in valid_names]
    valid_dest = "data/valid/"+waste_type
    move_files(valid_source_files,valid_dest)
    
    ## verileri test`e gonder
    test_names = get_names(waste_type,test_ind)
    test_source_files = [os.path.join(source_folder,name) for name in test_names]
    ## test folder at fn
    move_files(test_source_files,"data/test")


path = Path(os.getcwd())/"data"
path

# tfms = get_transforms(do_flip=True,flip_vert=True) --------------------------- HATA
data = ImageDataBunch.from_folder(path,test="test",ds_tfms=tfms,bs=16)
data
print(data.classes)
data.show_batch(rows=4,figsize=(10,8))

learn = create_cnn(data,models.resnet34,metrics=error_rate)
learn.model
learn.lr_find(start_lr=1e-6,end_lr=1e1)
learn.recorder.plot()
learn.fit_one_cycle(20,max_lr=5.13e-03)
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
interp.plot_top_losses(9, figsize=(15,11))
doc(interp.plot_top_losses)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
interp.most_confused(min_val=2)
preds = learn.get_preds(ds_type=DatasetType.Test)
print(preds[0].shape)
preds[0]
data.classes
max_idxs = np.asarray(np.argmax(preds[0],axis=1))
yhat = []
for max_idx in max_idxs:
    yhat.append(data.classes[max_idx])
yhat
learn.data.test_ds[0][0]
y = []

## POSIX yollarini string yap
for label_path in data.test_ds.items:
    y.append(str(label_path))
    
## atik turunu dosya yolundan cikar
pattern = re.compile("([a-z]+)[0-9]+")
for i in range(len(y)):
    y[i] = pattern.search(y[i]).group(1)
print(yhat[0:5])
print(y[0:5])
learn.data.test_ds[0][0]

cm = confusion_matrix(y,yhat)
print(cm)
df_cm = pd.DataFrame(cm,waste_types,waste_types)

plt.figure(figsize=(10,8))
sns.heatmap(df_cm,annot=True,fmt="d",cmap="YlGnBu")

correct = 0

for r in range(len(cm)):
    for c in range(len(cm)):
        if (r==c):
            correct += cm[r,c]
accuracy = correct/sum(sum(cm))
accuracy

## yer acmak icin herseyi siler
shutil.rmtree("data")
shutil.rmtree('atik-veriler')